In [10]:
from tqdm import tqdm
import os
import numpy as np
from pytorch3d.io import load_objs_as_meshes
from pathlib import Path
import pickle
data_pth = Path('/media/aaron/work/KIRI/ME_1015')
mean_vert = np.zeros(21918,)

def load_obj_vertex_data(obj_file_path):
    # 使用 PyTorch3D 来加载 .obj 文件
    meshes = load_objs_as_meshes([obj_file_path])
    vertices = meshes.verts_packed().detach().cpu().numpy()
    return vertices
cnt = 0
for category_folder in tqdm(os.listdir(str(data_pth))):
    category_path = data_pth / category_folder
    obj_file = category_path / 'model_align.obj'
    obj = load_obj_vertex_data(obj_file).flatten()
    mean_vert += obj
    cnt += 1
    
mean_vert /= cnt

  0%|          | 0/18 [00:00<?, ?it/s]/home/aaron/miniconda3/envs/mp/lib/python3.10/site-packages/pytorch3d/io/obj_io.py:544: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")
100%|██████████| 18/18 [00:01<00:00, 11.43it/s]


In [1]:
"""Default Hyperparameter configuration"""

import ml_collections


def get_config():
    """Get the default Hypermarameter configuration"""
    config = ml_collections.ConfigDict()
    config.model = 'Classic_CNN'
    config.dataset = 'multiface'
    
    config.data_size = 15250
    config.image_size = (3024, 4032)
    config.render_size = (512, 334)
    config.texture_size = (1024, 1024)
    
    config.num_epochs = 2000
    config.warmup_epochs = 1
    config.batch_size = 16
    config.steps_per_epoch = config.data_size // config.batch_size
    config.learning_rate = 1e-3
    config.log_every_steps = 10
    config.vertex_num = 7306
    config.num_train_steps = -1
    config.kpt_num = 478
    config.pca = '../training_data/pca_for_jax.pickle'
    config.mean_mesh = '../training_data/mean_mesh_for_jax.pickle'
    return config

In [2]:
config=get_config()

In [3]:
import tensorflow as tf
import numpy as np
import ml_collections
from absl import logging
from pathlib import Path

def readTFRECORD(tfrecord_pth: str,
                 config: ml_collections.ConfigDict) -> tf.data:

    global IMAGE_WIDTH, IMAGE_HEIGHT, NUM_VERTEX

    AUTOTUNE = tf.data.experimental.AUTOTUNE
    data_set = tf.data.TFRecordDataset(tfrecord_pth)


    IMAGE_WIDTH = config.render_size[1]
    IMAGE_HEIGHT = config.render_size[0]
    NUM_VERTEX = config.vertex_num

    data_set = (data_set
            .shuffle(config.data_size, reshuffle_each_iteration=True, seed=42)
            .map(parse, num_parallel_calls=AUTOTUNE)
            .repeat()
            .batch(config.batch_size, drop_remainder=True)
            # .cache()
            # .map(augment_using_ops,  num_parallel_calls=AUTOTUNE)
            .prefetch(buffer_size=AUTOTUNE)
        )


    return data_set


def augment_using_ops(batch):

    # batch['img'] = tf.image.random_brightness(batch['img'], 0.2)
    # batch['img'] = tf.keras.layers.RandomZoom(0.2)(batch['img'])
    # batch['img'] = tf.keras.layers.RandomRotation(0.1)(batch['img'])
    
    # images = tf.image.rot90(images)
    return batch


def parse(example_proto):
    features = tf.io.parse_single_example(
        example_proto,
        features={
            'img': tf.io.FixedLenFeature([], tf.string),
            'vtx': tf.io.FixedLenFeature([], tf.string),

        })

    img = features['img']
    vtx = features['vtx']


    img = tf.io.decode_raw(img, tf.uint8)
    img = tf.cast(img, tf.float32)
    vtx = tf.io.decode_raw(vtx, tf.float32)

    img = tf.reshape(img, [IMAGE_HEIGHT, IMAGE_WIDTH, 3])
    img = tf.image.rgb_to_grayscale(img)
    # img = (img - image_mean) / image_std
    img /= 255.
    vtx = tf.reshape(vtx, [NUM_VERTEX*3])
    return {
        'img': img,
        'vtx': vtx,

    }


2023-10-26 11:31:32.401523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 11:31:32.469388: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-26 11:31:32.840224: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.8/extras/CUPTI/lib64
2023-10-26 11:31:32.840269: W 

In [4]:
readTFRECORD(tfrecord_pth='/media/aaron/work/ITRI_SSTC/S100/FY112_FRP/code/FRP/training_data/1015_18pose_664nums.tfrecord', config=config)

2023-10-26 11:32:05.830216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-26 11:32:05.830344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-26 11:32:05.845549: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-26 11:32:05.845691: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-26 11:32:05.845759: I tensorflow/compiler/xla/stream_executo

<PrefetchDataset element_spec={'img': TensorSpec(shape=(16, 512, 334, 1), dtype=tf.float32, name=None), 'vtx': TensorSpec(shape=(16, 21918), dtype=tf.float32, name=None)}>